<a href="https://colab.research.google.com/github/me1nna/fake-image-detection/blob/main/Gradb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Градиентный бустинг + ембеддинги

Установка зависимостей


In [1]:
import torch
import numpy as np

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [2]:
import random
import os

import pickle
import numpy as np
from skimage import io

from tqdm import tqdm, tqdm_notebook
from PIL import Image
from pathlib import Path

from torchvision.transforms import v2
from torchvision import transforms
from multiprocessing.pool import ThreadPool
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader, Subset
import torch.nn as nn

from matplotlib import colors, pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings(action='ignore', category=DeprecationWarning)

In [3]:
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(42)
random.seed(42)
os.environ['PYTHONHASHSEED'] = "42"

In [4]:
import PIL
print(PIL.__version__)

9.4.0


In [ ]:

!gdown --id 13rZ1HNOWsN-cTYgYHu0S3UYUHI0KAUBV
!gdown --id 1qjK-uwy20OAWCO1l39RNvjRgjsZiFqm0
!gdown --id 1Heeu9E9vI3MtkYE-_iZdGGcytm5Mw8ha
!gdown --id 1vnG3dlMb9wxo0P-noc84LUBaZNjwumMq
!gdown --id 1BW2ARIM2r0EgL5Z4A8E0O4PXB2BNRhmA

!unzip "images.zip"

In [6]:
import seaborn as sns
import pandas as pd

In [7]:
data = pd.read_csv("train.csv")
data_test = pd.read_csv("test.csv")

Некоторые файлы в датасете не имеют расширения. Нетрудно заметить, если расширения не указано, то оно .png

In [8]:
def load_images(image_paths):
    images = []
    for image_path in image_paths:
        full_path = os.path.join('/content/images', image_path)
        if not os.path.isfile(full_path):
            full_path += ".png"
        try:
            images.append(full_path)
        except Exception as e:
            print(f"Не удалось загрузить изображение: {full_path}")
            print(f"Ошибка: {e}")
            continue
    return images

In [9]:
train_files = load_images(data["id"])
test_files = load_images(data_test["id"])

In [10]:
train_labels = data["target"]

In [11]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-o8ttulib
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-o8ttulib
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 555.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [12]:
import clip
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

# Загрузка модели CLIP
model, preprocess = clip.load('ViT-B/32', device='cuda')

# Подготовка изображений и получение эмбеддингов
def get_image_embeddings(image_paths):
    embeddings = []
    for path in image_paths:
        image = preprocess(Image.open(path)).unsqueeze(0).to('cuda')
        with torch.no_grad():
            embedding = model.encode_image(image).cpu().numpy()
        embeddings.append(embedding)
    return np.vstack(embeddings)


train_embeddings = get_image_embeddings(train_files)
test_embeddings = get_image_embeddings(test_files)

100%|████████████████████████████████████████| 338M/338M [00:01<00:00, 188MiB/s]


In [13]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 9.3 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostClassifier

cb_model = CatBoostClassifier(iterations=10000, max_depth=5, learning_rate=0.1, verbose=1)

cb_model.fit(train_embeddings, train_labels)



In [16]:
probs = cb_model.predict_proba(test_embeddings)[:,1]

my_submit = pd.read_csv("sample_submission.csv")

my_submit = pd.DataFrame({"id": my_submit["id"], "target": probs})
my_submit

,id,target
0,Qt1fGUB0Vz.jpeg,0.000010
1,j4Rhioq7R3.jpeg,0.318364
2,rD0hgFHJUZ.jpeg,0.745467
3,aY5z1EJsJ6.jpeg,0.999980
4,qZ3IoxD2TE.jpeg,0.000554
...,...,...
501,Qrme6EUmL1,0.998875
502,4369Dl8wfs.jpeg,0.323496
503,BrNEvRNpPu.jpeg,0.000134
504,DuwXCNmeMx.webp,0.476280


In [17]:
my_submit.to_csv("cat3.csv", index=False)

In [ ]:
from catboost import CatBoostClassifier

cb_model = CatBoostClassifier(iterations=15000, max_depth=3, learning_rate=0.1, verbose=1)

cb_model.fit(train_embeddings, train_labels)

In [19]:
probs = cb_model.predict_proba(test_embeddings)[:,1]

my_submit = pd.read_csv("sample_submission.csv")

my_submit = pd.DataFrame({"id": my_submit["id"], "target": probs})
my_submit

,id,target
0,Qt1fGUB0Vz.jpeg,0.000013
1,j4Rhioq7R3.jpeg,0.281531
2,rD0hgFHJUZ.jpeg,0.811748
3,aY5z1EJsJ6.jpeg,0.999948
4,qZ3IoxD2TE.jpeg,0.000504
...,...,...
501,Qrme6EUmL1,0.998860
502,4369Dl8wfs.jpeg,0.288669
503,BrNEvRNpPu.jpeg,0.000604
504,DuwXCNmeMx.webp,0.483962


In [20]:
my_submit.to_csv("cat4.csv", index=False)

In [ ]:
cb_model = CatBoostClassifier(iterations=20000, max_depth=2, learning_rate=0.1, verbose=1)

cb_model.fit(train_embeddings, train_labels)

In [22]:
probs = cb_model.predict_proba(test_embeddings)[:,1]

my_submit = pd.read_csv("sample_submission.csv")

my_submit = pd.DataFrame({"id": my_submit["id"], "target": probs})
my_submit

,id,target
0,Qt1fGUB0Vz.jpeg,0.000023
1,j4Rhioq7R3.jpeg,0.098202
2,rD0hgFHJUZ.jpeg,0.706146
3,aY5z1EJsJ6.jpeg,0.999949
4,qZ3IoxD2TE.jpeg,0.000833
...,...,...
501,Qrme6EUmL1,0.999557
502,4369Dl8wfs.jpeg,0.387923
503,BrNEvRNpPu.jpeg,0.000648
504,DuwXCNmeMx.webp,0.508681


In [ ]:
cb_model = CatBoostClassifier(iterations=20000, max_depth=3, learning_rate=0.1, verbose=1)

cb_model.fit(train_embeddings, train_labels)

In [24]:
probs = cb_model.predict_proba(test_embeddings)[:,1]

my_submit = pd.read_csv("sample_submission.csv")

my_submit = pd.DataFrame({"id": my_submit["id"], "target": probs})
my_submit

,id,target
0,Qt1fGUB0Vz.jpeg,0.000013
1,j4Rhioq7R3.jpeg,0.281680
2,rD0hgFHJUZ.jpeg,0.810168
3,aY5z1EJsJ6.jpeg,0.999950
4,qZ3IoxD2TE.jpeg,0.000459
...,...,...
501,Qrme6EUmL1,0.998911
502,4369Dl8wfs.jpeg,0.282310
503,BrNEvRNpPu.jpeg,0.000576
504,DuwXCNmeMx.webp,0.485471


In [25]:
my_submit.to_csv("cat5.csv", index=False)

In [26]:
cb_model = CatBoostClassifier(iterations=35000, max_depth=1, learning_rate=0.1, verbose=1)

cb_model.fit(train_embeddings, train_labels)

Выходные данные были обрезаны до нескольких последних строк (5000).
30000:	learn: 0.0022118	total: 3m 58s	remaining: 39.7s
30001:	learn: 0.0022119	total: 3m 58s	remaining: 39.7s
30002:	learn: 0.0022118	total: 3m 58s	remaining: 39.7s
30003:	learn: 0.0022118	total: 3m 58s	remaining: 39.7s
30004:	learn: 0.0022118	total: 3m 58s	remaining: 39.6s
30005:	learn: 0.0022118	total: 3m 58s	remaining: 39.6s
30006:	learn: 0.0022118	total: 3m 58s	remaining: 39.6s
30007:	learn: 0.0022118	total: 3m 58s	remaining: 39.6s
30008:	learn: 0.0022118	total: 3m 58s	remaining: 39.6s
30009:	learn: 0.0022118	total: 3m 58s	remaining: 39.6s
30010:	learn: 0.0022118	total: 3m 58s	remaining: 39.6s
30011:	learn: 0.0022117	total: 3m 58s	remaining: 39.6s
30012:	learn: 0.0022117	total: 3m 58s	remaining: 39.6s
30013:	learn: 0.0022117	total: 3m 58s	remaining: 39.6s
30014:	learn: 0.0022117	total: 3m 58s	remaining: 39.6s
30015:	learn: 0.0022118	total: 3m 58s	remaining: 39.6s
30016:	learn: 0.0022117	total: 3m 58s	remaining: 39.

In [27]:
probs = cb_model.predict_proba(test_embeddings)[:,1]

my_submit = pd.read_csv("sample_submission.csv")

my_submit = pd.DataFrame({"id": my_submit["id"], "target": probs})
my_submit

,id,target
0,Qt1fGUB0Vz.jpeg,0.000026
1,j4Rhioq7R3.jpeg,0.236849
2,rD0hgFHJUZ.jpeg,0.667630
3,aY5z1EJsJ6.jpeg,0.999965
4,qZ3IoxD2TE.jpeg,0.000604
...,...,...
501,Qrme6EUmL1,0.999777
502,4369Dl8wfs.jpeg,0.257456
503,BrNEvRNpPu.jpeg,0.000676
504,DuwXCNmeMx.webp,0.627391


In [ ]:
my_submit.to_csv("cat6.csv", index=False)

In [28]:
cb_model = CatBoostClassifier(iterations=50000, max_depth=3, learning_rate=0.1, verbose=1)

cb_model.fit(train_embeddings, train_labels)

Выходные данные были обрезаны до нескольких последних строк (5000).
45000:	learn: 0.0007824	total: 20m 29s	remaining: 2m 16s
45001:	learn: 0.0007824	total: 20m 29s	remaining: 2m 16s
45002:	learn: 0.0007824	total: 20m 29s	remaining: 2m 16s
45003:	learn: 0.0007823	total: 20m 29s	remaining: 2m 16s
45004:	learn: 0.0007824	total: 20m 29s	remaining: 2m 16s
45005:	learn: 0.0007823	total: 20m 29s	remaining: 2m 16s
45006:	learn: 0.0007823	total: 20m 29s	remaining: 2m 16s
45007:	learn: 0.0007824	total: 20m 29s	remaining: 2m 16s
45008:	learn: 0.0007824	total: 20m 30s	remaining: 2m 16s
45009:	learn: 0.0007823	total: 20m 30s	remaining: 2m 16s
45010:	learn: 0.0007824	total: 20m 30s	remaining: 2m 16s
45011:	learn: 0.0007823	total: 20m 30s	remaining: 2m 16s
45012:	learn: 0.0007824	total: 20m 30s	remaining: 2m 16s
45013:	learn: 0.0007823	total: 20m 30s	remaining: 2m 16s
45014:	learn: 0.0007823	total: 20m 30s	remaining: 2m 16s
45015:	learn: 0.0007823	total: 20m 30s	remaining: 2m 16s
45016:	learn: 0.0007

In [29]:
probs = cb_model.predict_proba(test_embeddings)[:,1]

my_submit = pd.read_csv("sample_submission.csv")

my_submit = pd.DataFrame({"id": my_submit["id"], "target": probs})
my_submit

,id,target
0,Qt1fGUB0Vz.jpeg,0.000008
1,j4Rhioq7R3.jpeg,0.259695
2,rD0hgFHJUZ.jpeg,0.792460
3,aY5z1EJsJ6.jpeg,0.999968
4,qZ3IoxD2TE.jpeg,0.000287
...,...,...
501,Qrme6EUmL1,0.999164
502,4369Dl8wfs.jpeg,0.247742
503,BrNEvRNpPu.jpeg,0.000445
504,DuwXCNmeMx.webp,0.503560


In [30]:
my_submit.to_csv("cat_crazy.csv", index=False)